# BBDD Relacionales
## SQLite3


In [22]:
import sqlite3
from sqlite3 import Error

from datetime import datetime



In [23]:
def create_connection(db_file):
    """ Crea una conexión a una Base de Datos SQLite Local
        :parametros db_file: base de datos
        :return: el objeto de la conexión a la BBDD o None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
        if conn:
            conn.close()
    return conn

def create_connection_memory():
    """ Crea una conexión a una Base de Datos SQLite en Memoria"""
    conn = None;
    try:
        conn = sqlite3.connect(':memory:')
        print(sqlite3.version)
    except Error as e:
        print(e)
        if conn:
            conn.close()
    return conn

def create_table(conn, create_table_sql):
    """ Crear una Table en una BBDD SQLite
    :parametro conn: Conexión a la base de datos SQLite
    :parametro create_table_sql: La instrucción CREATE TABLE
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

def create_project(conn, project):
    """
    Inserta un nuevo poryecto en la Tabla Proyectos
    :param conn: Conexión a la base de datos SQLite
    :param project:
    :return: project id
    """
    sql = ''' INSERT INTO proyectos(nombre, f_inicio, f_fin, lider_proyecto, presupuesto)
              VALUES(?,?,?,?,?) '''
    
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

def create_task(conn, task):
    """
    Inserta una actividad o tarea en la Tabla actividades
    :param conn: Conexión a la base de datos SQLite
    :param task:
    :return:
    """

    sql = ''' INSERT INTO actividades(nombre_tarea,prioridad,estatus,id_proyecto,f_inicio_t,f_fin_t)
              VALUES(?,?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()

    return cur.lastrowid


def update_task(conn, task):
    """
    Actualiza prioridad, fecha de inicio, fecha fin de una actividad
    :param conn: Conexión a la base de datos SQLite
    :param task:
    :return: project id
    """
    sql = ''' UPDATE actividades
              SET prioridad = ? ,
                  f_inicio_t = ? ,
                  f_fin_t = ?
              WHERE id = ?'''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()

def select_all_tasks(conn):
    """
    Muestra todas las actividades de la tabla actividades
    :param conn: Conexión a la base de datos SQLite
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM actividades")

    rows = cur.fetchall()

    for row in rows:
        print(row)
        
def select_task_by_priority(conn, priority):
    """
    Mostrar todas las actividades por prioridad
    :param conn: Conexión a la base de datos SQLite
    :param priority:
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM actividades WHERE prioridad=?", (priority,))

    rows = cur.fetchall()

    for row in rows:
        print(row)
        
def delete_task(conn, id):
    """
    Borra una actividad por id
    :param conn: Conexión a la base de datos SQLite 
    :param id: id de la tarea
    :return:
    """
    sql = 'DELETE FROM actividades WHERE id=?'
    cur = conn.cursor()
    cur.execute(sql, (id,))
    conn.commit()
    
def delete_all_tasks(conn):
    """
    Borra todas las actividades 
    :param conn: Conexión a la base de datos SQLite
    :return:
    """
    sql = 'DELETE FROM actividades'
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()
    
def sql_fetch(conn):
    """
    Muestra todas las tablas de la BBDD
    """
    cursorObj = conn.cursor()
    cursorObj.execute('SELECT name from sqlite_master where type= "table"')
    print(cursorObj.fetchall())



In [24]:
if __name__ == '__main__':
    
    database = r'C:\Users\msierra\Desktop\SQLDatabase\Proyectos3.db'
    
    # crear la conexión a la base de datos local
    conn = create_connection(database)
    
    # crear la conexión a la base de datos en memoria
    #create_connection_memory()
       
    # crear las tablas
    
    
    sql_create_projects_table = """ CREATE TABLE IF NOT EXISTS proyectos (
                                        id integer PRIMARY KEY,
                                        nombre text NOT NULL,
                                        f_inicio text,
                                        f_fin text,
                                        lider_proyecto text,
                                        presupuesto float
                                    ); """

    sql_create_tasks_table = """CREATE TABLE IF NOT EXISTS actividades (
                                    id integer PRIMARY KEY,
                                    nombre_tarea text NOT NULL,
                                    prioridad integer,
                                    estatus texto NOT NULL,
                                    id_proyecto integer NOT NULL,
                                    f_inicio_t text NOT NULL,
                                    f_fin_t text NOT NULL,
                                    FOREIGN KEY (id_proyecto) REFERENCES proyectos (id)
                                );"""
    
    
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_projects_table)

        # create tasks table
        create_table(conn, sql_create_tasks_table)
    else:
        print("Error! La conexión a la BBDD no ha sido creada...")
        
    # create a new project
    project = ( 'PC Refresh', datetime(2020,1,1), datetime(2020,12,31), "Maria Sierra", 125000.00);
    project_id = create_project(conn, project)    

2.6.0


In [26]:
sql_fetch(conn)

[('proyectos',), ('actividades',)]


In [27]:
cursor = conn.cursor()
cursor.execute('SELECT * FROM proyectos ')

rows = cursor.fetchall()
for row in rows:
    print(row)

(1, 'PC Refresh', '2020-01-01 00:00:00', '2020-12-31 00:00:00', 'Maria Sierra', 125000.0)


In [28]:
# tasks
task_1 = ('Realizar inventarios de los equipo disponibles', 1, "En proceso", project_id, '2015-01-01', '2015-01-02')
task_2 = ('Obtener listado de nuevos ingresos', 1, "En proceso", project_id, '2015-01-03', '2015-01-05')

# create tasks
create_task(conn, task_1)
create_task(conn, task_2)

2

In [29]:
task_3 = ('Obtener listado de nuevos ingresos', 1, "En proceso", project_id, '2015-01-03', '2015-01-05')
create_task(conn, task_3)

3

In [30]:
task_4 = ('Obtener listado', 2, "En proceso", project_id, '2015-01-03', '2015-01-05')
create_task(conn, task_4)

4

In [31]:
print("Mostrar todas las tareas")
select_all_tasks(conn)

Mostrar todas las tareas
(1, 'Realizar inventarios de los equipo disponibles', 1, 'En proceso', 1, '2015-01-01', '2015-01-02')
(2, 'Obtener listado de nuevos ingresos', 1, 'En proceso', 1, '2015-01-03', '2015-01-05')
(3, 'Obtener listado de nuevos ingresos', 1, 'En proceso', 1, '2015-01-03', '2015-01-05')
(4, 'Obtener listado', 2, 'En proceso', 1, '2015-01-03', '2015-01-05')


In [32]:
print("Consultar las tareas por prioridad")
select_task_by_priority(conn, 1)

Consultar las tareas por prioridad
(1, 'Realizar inventarios de los equipo disponibles', 1, 'En proceso', 1, '2015-01-01', '2015-01-02')
(2, 'Obtener listado de nuevos ingresos', 1, 'En proceso', 1, '2015-01-03', '2015-01-05')
(3, 'Obtener listado de nuevos ingresos', 1, 'En proceso', 1, '2015-01-03', '2015-01-05')


In [35]:
consulta = '''SELECT Proyectos.nombre, Proyectos.f_inicio, Proyectos.f_fin, Actividades.nombre_tarea, Actividades.estatus
FROM Proyectos INNER JOIN Actividades ON Proyectos.id = Actividades.id_proyecto;'''

cursor = conn.cursor()
cursor.execute(consulta)

rows = cursor.fetchall()
proyecto = ""

for row in rows:
    if proyecto == "" or proyecto != row[0]:
        print("Proyecto: ", row[0])
        print("Fecha Inicio: ", datetime.strptime(row[1], "%Y-%m-%d %H:%M:%S"))
        print("Fecha Fin: ", datetime.strptime(row[2], "%Y-%m-%d %H:%M:%S"))
        print("")
        print("---------------- Actividades ------------------\n")
        proyecto = row[0]
    else:
        print(f"{row[3]} - {row[4]} ")

Proyecto:  PC Refresh
Fecha Inicio:  2020-01-01 00:00:00
Fecha Fin:  2020-12-31 00:00:00

---------------- Actividades ------------------

Obtener listado de nuevos ingresos - En proceso 
Obtener listado de nuevos ingresos - En proceso 
Obtener listado - En proceso 


In [ ]:
# Eliminar actividades
delete_task(conn, 2)

In [ ]:
# Eliminar todas las actividades
delete_all_tasks(conn)

In [13]:
conn.close()